In [19]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Iterable

DEFAULT_SKIP_DIRS = {
    ".git", ".hg", ".svn",
    "__pycache__", ".pytest_cache", ".mypy_cache", ".ruff_cache",
    ".venv", "venv", "env",
    "node_modules",
    ".idea", ".vscode",
    ".ipynb_checkpoints",
    "dist", "build", ".build",
}

# Endret: Lagt til "ALL.ipynb" her slik at den ignoreres overalt
DEFAULT_SKIP_FILES = {".DS_Store", "ALL.ipynb"}


def detect_root_dir() -> Path:
    """Root = mappen scriptet ligger i. I Jupyter: current working directory."""
    if "__file__" in globals():
        return Path(__file__).resolve().parent
    return Path.cwd().resolve()


def iter_paths(root: Path) -> Iterable[Path]:
    """Yield alle paths under root, med skip-regler."""
    for p in root.rglob("*"):
        if any(part in DEFAULT_SKIP_DIRS for part in p.parts):
            continue
        # Sjekker om filnavnet er i DEFAULT_SKIP_FILES (inkl. ALL.ipynb)
        if p.is_file() and p.name in DEFAULT_SKIP_FILES:
            continue
        yield p


def tree_lines(root: Path) -> list[str]:
    lines = []
    lines.append("=== DIRECTORY TREE ===")
    lines.append(str(root.resolve()))
    lines.append("")

    rels = sorted([p.relative_to(root) for p in iter_paths(root)], key=lambda x: str(x).lower())

    for rel in rels:
        indent = "  " * (len(rel.parts) - 1)
        lines.append(f"{indent}- {rel.name}")

    lines.append("")
    return lines


def extract_ipynb_text_no_outputs(ipynb_path: Path) -> str:
    """
    Notebook -> tekst uten outputs:
    - Markdown-celler kommenteres
    - Code-celler: kun source
    - outputs/execution_count ignoreres alltid
    """
    # Prøv nbformat først (robust), fallback til raw json
    nb = None
    try:
        import nbformat  # type: ignore
        nb = nbformat.read(ipynb_path, as_version=4)
    except Exception:
        nb = json.loads(ipynb_path.read_text(encoding="utf-8", errors="replace"))

    out: list[str] = []
    cells = nb.get("cells", [])

    for i, cell in enumerate(cells, start=1):
        ctype = cell.get("cell_type", "unknown")
        source = cell.get("source", [])
        if isinstance(source, str):
            source = [source]

        out.append(f"\n# === CELL {i} ({ctype}) ===\n")

        if ctype == "markdown":
            for s in source:
                s = s.rstrip("\n")
                out.append("# " + s + "\n" if s.strip() else "#\n")
        elif ctype == "code":
            for s in source:
                out.append(s if s.endswith("\n") else s + "\n")
        else:
            for s in source:
                s = s.rstrip("\n")
                out.append("# " + s + "\n")

    return "".join(out).lstrip("\n")


def ipynb_contents_lines(root: Path) -> list[str]:
    """Kun innhold av .ipynb-filer."""
    lines: list[str] = []
    lines.append("=== NOTEBOOK CONTENTS (.ipynb only) ===")
    lines.append("")

    notebooks = sorted(
        [p for p in iter_paths(root) if p.is_file() and p.suffix.lower() == ".ipynb"],
        key=lambda p: str(p.relative_to(root)).lower(),
    )

    for nb_path in notebooks:
        rel = nb_path.relative_to(root)
        lines.append(f"\n--- NOTEBOOK: {rel} (outputs STRIPPED) ---\n")
        try:
            lines.append(extract_ipynb_text_no_outputs(nb_path))
        except Exception as e:
            lines.append(f"[ERROR reading ipynb: {e}]")

    lines.append("")
    return lines


def dump_repo(output_filename: str = "repo_dump.txt") -> Path:
    root = detect_root_dir()
    out_path = root / output_filename

    lines: list[str] = []
    lines.extend(tree_lines(root))
    lines.extend(ipynb_contents_lines(root))

    out_path.write_text("\n".join(lines), encoding="utf-8")
    return out_path


# Kjør dump
if __name__ == "__main__":
    out_file = dump_repo("repo_dump.txt")
    print(f"Saved dump to: {out_file}")

Saved dump to: /home/kristoffny/ICT_final/repo_dump.txt


In [20]:
import os
import zipfile
from pathlib import Path

# --- KONFIGURASJON ---
SOURCE_DIR = Path("./artifacts")
OUTPUT_ZIP = "samlede_plott_med_kontekst.zip"
OUTPUT_TXT = "plott_beskrivelser.txt"

def collect_and_rename_plots():
    if not SOURCE_DIR.exists():
        print(f"Fant ikke mappen: {SOURCE_DIR}")
        return

    print(f"Starter innsamling fra {SOURCE_DIR}...")
    
    files_processed = 0
    
    # Åpne både zip-filen og tekstfilen for skriving
    with zipfile.ZipFile(OUTPUT_ZIP, 'w', compression=zipfile.ZIP_DEFLATED) as zf, \
         open(OUTPUT_TXT, 'w', encoding='utf-8') as log_file:
        
        # Skriv header i tekstfilen
        log_file.write(f"=== OVERSIKT OVER PLOTT ===\n")
        log_file.write(f"Kilde: {SOURCE_DIR.resolve()}\n")
        log_file.write(f"{'='*80}\n\n")
        log_file.write(f"{'NYTT FILNAVN (I ZIP)':<70} | {'OPPRINNELIG STI'}\n")
        log_file.write(f"{'-'*70} | {'-'*40}\n")

        # Gå rekursivt gjennom alle mapper
        for file_path in sorted(SOURCE_DIR.rglob("*.png")):
            # Finn den relative stien fra artifacts-mappen
            # Eks: 03_exp2_sbls/run_300ep/loss_curve.png
            rel_path = file_path.relative_to(SOURCE_DIR)
            
            # Lag nytt navn ved å erstatte mappeseparator med dobbel understrek
            # Dette gjør at mappestrukturen blir bakt inn i filnavnet
            # Eks: 03_exp2_sbls__run_300ep__loss_curve.png
            new_name = str(rel_path).replace(os.sep, "__")
            
            # Legg filen inn i ZIP-arkivet med det nye navnet
            zf.write(file_path, arcname=new_name)
            
            # Skriv til loggfilen
            log_file.write(f"{new_name:<70} | {rel_path}\n")
            
            files_processed += 1

    print(f"\nFerdig!")
    print(f"-> {files_processed} bilder er lagt til i '{OUTPUT_ZIP}'")
    print(f"-> Detaljert oversikt er lagret i '{OUTPUT_TXT}'")

if __name__ == "__main__":
    collect_and_rename_plots()

Starter innsamling fra artifacts...

Ferdig!
-> 89 bilder er lagt til i 'samlede_plott_med_kontekst.zip'
-> Detaljert oversikt er lagret i 'plott_beskrivelser.txt'
